# Set initialization file
https://cccma.gitlab.io/classic/makeInputFiles.html

Initialize the model for CLASSIC (CLASS+CTEM) run.

In [1]:
# Env: sc2_v0

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt # New plot library (https://proplot.readthedocs.io/en/latest/)
pplt.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures

In [2]:
exp = 'Ref'
site = 'byl'
site_ex = 'CA-Oas'

path_in = '/home/lalandmi/eccc/classic-develop/inputFiles/FLUXNETsites/'+site_ex # example file
path_out = '/home/lalandmi/eccc/classic-develop/inputFiles/SnowArctic/'+site

In [3]:
ds = xr.open_dataset(path_in+'/'+site_ex+'_init.nc')
ds.load()

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 53.63
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * lon                (lon) float64 -106.2
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 15.16 20.08 ... 23.29
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

In [4]:
ds.soilcmas.sum()

<xarray.DataArray 'soilcmas' ()>
array(0.)

## Bylot Island, Canadian high Arctic

The site is a low-center polygon, with herb tundra and no erect vegetation

| Short name | byl |
|:-----------|:----|
| Location | 73.1504ºN, 80.0046ºW |
| Elevation | 25 m |
| Snow-free albedo | - |
| Simulation period | 11 July 2013 to 25 June 2019 |
| Temperature/humidity measurement height | 2.3 m |
| Wind measurement height | 2.3 m |
| Reference | Domine et al. ([2021](https://essd.copernicus.org/articles/13/4331/2021/)) |

In [5]:
lat = 73.1504
lon = -80.0046

# If the site description is not enough we can get the input data from gridded satellite datasets
path = '/home/lalandmi/Dropbox/data/CLASSIC/'
SoilGrids250m_CLAY = xr.open_dataset(path+'/soil/SoilGrids250m_CLAY_0.05deg.nc')
SoilGrids250m_ORGM = xr.open_dataset(path+'/soil/SoilGrids250m_ORGM_0.05deg.nc')
SoilGrids250m_SAND = xr.open_dataset(path+'/soil/SoilGrids250m_SAND_0.05deg.nc')
SoilGrids250m_SDEP = xr.open_dataset(path+'/soil/SoilGrids250m_SDEP_0.05deg.nc')
ESACCI_PFT_2000 = xr.open_dataset(path+'/ESACCI-LC-L4-PFT-Map-300m-P1Y-2000-v2.0.8.nc')
ESACCI_PFT_2010 = xr.open_dataset(path+'/ESACCI-LC-L4-PFT-Map-300m-P1Y-2010-v2.0.8.nc')
CTEM_12_PFTs_ESACCI_2010 = xr.open_dataset(path+'/CTEM_12_PFTs_ESACCI_2010_minus_water_rescaled_v2_1deg.nc')
NCAR_SOCI = xr.open_dataset(path+'/soil/NCAR_SOCI_0.5deg.nc')
c4_fraction_1deg = xr.open_dataarray(path+'/vegetation/c4_fraction_1deg.nc')

### Set the layer coords to the center soil layer

In [6]:
layer_c = []

for i in range(len(ds.DELZ)):
    if i == 0:
        layer_c.append(ds.DELZ.cumsum().values[i]/2)
    else:
        layer_c.append(ds.DELZ.cumsum().values[i-1] + ds.DELZ.values[i]/2)
        
with xr.set_options(keep_attrs=True):
    ds = ds.assign_coords(layer=ds.layer*0+layer_c)

ds.layer

<xarray.DataArray 'layer' (layer: 20)>
array([5.00e-02, 1.50e-01, 2.50e-01, 3.50e-01, 4.50e-01, 5.50e-01, 6.50e-01,
       7.50e-01, 8.50e-01, 9.50e-01, 1.10e+00, 1.35e+00, 1.70e+00, 2.15e+00,
       2.90e+00, 4.90e+00, 8.90e+00, 1.89e+01, 4.14e+01, 5.89e+01])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    standard_name:  ground layers

In [7]:
ds.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9

### Check soil data
https://essd.copernicus.org/articles/13/4331/2021/essd-13-4331-2021-supplement.pdf

Use information from Figure S1 and S2 + satellite data

#### SAND

In [8]:
SoilGrids250m_SAND.sel(lat=lat, lon=lon, method='nearest').SAND.values

array([37.06446 , 40.46604 , 42.833717, 45.39601 , 45.39601 , 45.39601 ,
       45.868103, 45.868103, 45.868103, 45.868103, 44.917828, 44.917828,
       44.917828, 44.917828, 44.917828, 44.917828, 44.917828, 44.917828,
       44.917828, 44.917828], dtype=float32)

In [22]:
SAND = [47.6, 43.5] + [63.5]*(20-2) # values reported from Figure S2

# The visibly (dark) organic-rich surface soil layer is 2 to 10 cm thick.
# -> could justify a peat layer in surface? (as the soil data where taken in a area with less moss than others)

# Average the last layers with satellite data
SAND[3:] = (SAND[3:] + SoilGrids250m_SAND.sel(lat=lat, lon=lon, method='nearest').SAND.values[3:])/2
SAND

[47.6,
 43.5,
 63.5,
 54.44800567626953,
 54.44800567626953,
 54.44800567626953,
 54.684051513671875,
 54.684051513671875,
 54.684051513671875,
 54.684051513671875,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586,
 54.208913803100586]

#### CLAY

In [17]:
SoilGrids250m_CLAY.sel(lat=lat, lon=lon, method='nearest').CLAY.values

array([12.033512, 13.197638, 13.949138, 14.984376, 14.984376, 14.984376,
       15.190086, 15.190086, 15.190086, 15.190086, 14.772641, 14.772641,
       14.772641, 14.772641, 14.772641, 14.772641, 14.772641, 14.772641,
       14.772641, 14.772641], dtype=float32)

In [23]:
CLAY = [0.0, 0.0] + [0.1]*(20-2) # values reported from Figure S2

# Average the last layers with satellite data
CLAY[3:] = (CLAY[3:] + SoilGrids250m_CLAY.sel(lat=lat, lon=lon, method='nearest').CLAY.values[3:])/2
CLAY

[0.0,
 0.0,
 0.1,
 7.542187976837158,
 7.542187976837158,
 7.542187976837158,
 7.645043182373047,
 7.645043182373047,
 7.645043182373047,
 7.645043182373047,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729,
 7.4363205909729]

#### ORGM

In [24]:
SoilGrids250m_ORGM.sel(lat=lat, lon=lon, method='nearest').ORGM.values

array([25.373564, 19.403736, 15.459769, 11.422414, 11.422414, 11.422414,
       10.646552, 10.646552, 10.646552, 10.646552, 10.014368, 10.014368,
       10.014368, 10.014368, 10.014368, 10.014368, 10.014368, 10.014368,
       10.014368, 10.014368], dtype=float32)

In [25]:
ORGM = [6.0, 1.5] + [1.5]*(20-2) # values approximatly reported from Figure S1
# There was less moss at this spot than in most other places on the polygon

# Average the last layers with satellite data
ORGM[3:] = (ORGM[3:] + SoilGrids250m_ORGM.sel(lat=lat, lon=lon, method='nearest').ORGM.values[3:])/2
ORGM

[6.0,
 1.5,
 1.5,
 6.461206912994385,
 6.461206912994385,
 6.461206912994385,
 6.073276042938232,
 6.073276042938232,
 6.073276042938232,
 6.073276042938232,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488,
 5.757184028625488]

In [26]:
ESACCI_PFT_2000.sel(lat=lat, lon=lon, method='nearest').load()

<xarray.Dataset>
Dimensions:       (time: 1, bounds: 2)
Coordinates:
    lon           float64 -80.0
    lat           float64 73.15
  * time          (time) datetime64[ns] 2000-01-01
Dimensions without coordinates: bounds
Data variables: (12/20)
    WATER         (time) int8 46
    BARE          (time) int8 0
    BUILT         (time) int8 0
    GRASS-MAN     (time) int8 0
    GRASS-NAT     (time) int8 37
    SHRUBS-BD     (time) int8 0
    ...            ...
    TREES-NE      (time) int8 17
    LAND          (time) int8 54
    WATER_OCEAN   (time) int8 0
    lat_bounds    (bounds) float64 73.15 73.15
    lon_bounds    (bounds) float64 -80.01 -80.0
    time_bounds   (time, bounds) datetime64[ns] 2000-01-01 2000-12-31
Attributes: (12/40)
    title:                      ESA CCI Land Cover Project: Maps of Plant Fun...
    summary:                    This dataset contains the global plant functi...
    type:                       ESACCI-LC-L4-PFT-Map-300m-P1Y
    id:                         ESACCI-LC-L4-PFT-Map-300m-P1Y-2000-v2.0.8
    project:                    Medium Resolution Land Cover - Climate Change...
    references:                 https://maps.elie.ucl.ac.be/CCI/viewer/downlo...
    ...                         ...
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  002777777777778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  002777777777778
    TileSize:                   2025:2025
    product_version:            2.0.8

In [27]:
ESACCI_PFT_2010.sel(lat=lat, lon=lon, method='nearest').load()

<xarray.Dataset>
Dimensions:       (time: 1, bounds: 2)
Coordinates:
    lon           float64 -80.0
    lat           float64 73.15
  * time          (time) datetime64[ns] 2010-01-01
Dimensions without coordinates: bounds
Data variables: (12/18)
    WATER         (time) int8 46
    BARE          (time) int8 0
    BUILT         (time) int8 0
    GRASS-MAN     (time) int8 0
    GRASS-NAT     (time) int8 37
    SHRUBS-BD     (time) int8 0
    ...            ...
    TREES-BE      (time) int8 0
    TREES-ND      (time) int8 0
    TREES-NE      (time) int8 17
    lat_bounds    (bounds) float64 73.15 73.15
    lon_bounds    (bounds) float64 -80.01 -80.0
    time_bounds   (time, bounds) datetime64[ns] 2010-01-01 2010-12-31
Attributes: (12/37)
    title:                      ESA CCI Land Cover Project: Maps of Plant Fun...
    summary:                    This dataset contains the global plant functi...
    type:                       ESACCI-LC-L4-PFT-Map-300m-P1Y
    id:                         ESACCI-LC-L4-PFT-Map-300m-P1Y-2010-v2.0.8
    project:                    Climate Change Initiative - European Space Ag...
    references:                 http://www.esa-landcover-cci.org
    ...                         ...
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  002777777777778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  002777777777778
    TileSize:                   2025:2025
    product_version:            2.0.8

In [28]:
CTEM_12_PFTs_ESACCI_2010.sel(lat=lat, lon=360+lon, method='nearest').load()

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    lon      float64 280.0
    lat      float64 73.5
Data variables: (12/13)
    NE       float32 0.0
    ND       float32 0.0
    BE       float32 0.0
    BCD      float64 0.0
    BDD      float64 0.0
    C3C      float32 0.0
    ...       ...
    C3G      float32 0.0
    C4G      float32 0.0
    Sedge    float32 0.06991
    SBE      float32 0.04521
    SBD      float32 0.04521
    BARE     float32 0.8397
Attributes:
    CDI:                       Climate Data Interface version 1.6.8 (http://m...
    Conventions:               CF-1.5
    cdo_openmp_thread_number:  8
    CDO:                       Climate Data Operators version 1.6.8rc2 (http:...
    created_by:                Ed Chan, CRD
    creation_date:             Wed Nov  9 19:52:31 GMT 2022
    provenance:                #1: https://climate.esa.int/en/projects/land-c...

In [29]:
# These values can be attributed with the global grid if not available
SDEP = SoilGrids250m_SDEP.sel(lat=lat, lon=lon, method='nearest').SDEP.values.item(0) # Soil permeable depth (m)
SOCI = NCAR_SOCI.sel(lat=lat, lon=lon, method='nearest').SOCI.values.item(0) # Soil color inde
c4_fraction = c4_fraction_1deg.sel(lat=lat, lon=lon, method='nearest').values.item(0) # If grass


# Those can be distributed along the soil layers if available
# CLAY = CLAY.values
# SAND = SAND.values
# ORGM = ORGM.values
# If the sum does not reach 100 %, the left over will be attributed to silt

# ! List of CLASS-level PFTs
classpfts = ['NdlTr', 'BdlTr', 'Crops', 'Grass', 'BdlSh']
FCAN = {'NdlTr': 0. , 'BdlTr': 0., 'Crops': 0., 'Grass': 1.0, 'BdlSh': 0.} # max 1
# Set grass to 0.7 (could be 0.5 or other) to let a little bare ground (as described for this site)

# ! List of CTEM PFTs
# ! **Note: 'BdlDCoTr' should be specified before 'BdlDDrTr' due to some code in competition.
ctempfts = ['NdlEvgTr', 'NdlDcdTr', 'BdlEvgTr', 'BdlDCoTr', 'BdlDDrTr', 'CropC3', 'CropC4', 'GrassC3', 'GrassC4', 'Sedge', 'BdlEvgSh', 'BdlDCoSh']
fcancmx = {'NdlEvgTr': 0., 'NdlDcdTr': 0., 'BdlEvgTr': 0., 'BdlDCoTr': 0., 'BdlDDrTr': 0., 'CropC3': 0., 'CropC4': 0., 
           'GrassC3': 1-c4_fraction, 'GrassC4': c4_fraction, 'Sedge': 0., 'BdlEvgSh': 0., 'BdlDCoSh': 0.} # Max 1
# Check differences between C3 and C4 at Cdp? (TODO)

if sum(FCAN.values()) > 1: raise Exception("The sum of FCAN values needs to be lower than 1.")
if sum(fcancmx.values()) > 1: raise Exception("The sum of fcanmax values needs to be lower than 1.")

print('SDEP = ' + str(SDEP))
print('\nSOCI = ' + str(SOCI))
print('\nCLAY = ' + str(CLAY))
print('\nSAND = ' + str(SAND))
print('\nORGM = ' + str(ORGM))
print('\nGrass C4 fraction = ' + str(c4_fraction))

SDEP = 4.960000038146973

SOCI = 6.0

CLAY = [0.0, 0.0, 0.1, 7.542187976837158, 7.542187976837158, 7.542187976837158, 7.645043182373047, 7.645043182373047, 7.645043182373047, 7.645043182373047, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729, 7.4363205909729]

SAND = [47.6, 43.5, 63.5, 54.44800567626953, 54.44800567626953, 54.44800567626953, 54.684051513671875, 54.684051513671875, 54.684051513671875, 54.684051513671875, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586, 54.208913803100586]

ORGM = [6.0, 1.5, 1.5, 6.461206912994385, 6.461206912994385, 6.461206912994385, 6.073276042938232, 6.073276042938232, 6.073276042938232, 6.073276042938232, 5.757184028625488, 5.757184028625488, 5.757184028625488, 5.757184028625488, 5.757184028625488, 5.757184028625488, 5

### Set lat/lon

In [30]:
with xr.set_options(keep_attrs=True):
    ds = ds.assign_coords(lat=(ds.lat*0+lat))
    ds = ds.assign_coords(lon=(ds.lon*0+lon))
ds

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 73.15
  * layer              (layer) float64 0.05 0.15 0.25 0.35 ... 18.9 41.4 58.9
  * lon                (lon) float64 -80.0
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 15.16 20.08 ... 23.29
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

### Set PFTs

In [31]:
for i, pft in enumerate(classpfts):
    ds.FCAN[0, i, 0, 0] = FCAN[pft]
    print(pft + ' -> ' + str(FCAN[pft]*100) + ' %')
    
ds.FCAN[0, :, 0, 0]
# 6th PFT: Bareground (need to be specified if CLASS used without CTEM)

NdlTr -> 0.0 %
BdlTr -> 0.0 %
Crops -> 0.0 %
Grass -> 100.0 %
BdlSh -> 0.0 %


<xarray.DataArray 'FCAN' (icp1: 6)>
array([0., 0., 0., 1., 0., 0.])
Coordinates:
  * icp1     (icp1) int32 1 2 3 4 5 6
    lat      float64 73.15
    lon      float64 -80.0
    tile     int32 1
Attributes:
    units:      -
    long_name:  Annual maximum fractional coverage of modelled area (read in ...

In [32]:
for i, pft in enumerate(ctempfts):
    ds.fcancmx[0, i, 0, 0] = fcancmx[pft]
    print(pft + ' -> ' + str(fcancmx[pft]*100) + ' %')
    
ds.fcancmx[0, :, 0, 0]

NdlEvgTr -> 0.0 %
NdlDcdTr -> 0.0 %
BdlEvgTr -> 0.0 %
BdlDCoTr -> 0.0 %
BdlDDrTr -> 0.0 %
CropC3 -> 0.0 %
CropC4 -> 0.0 %
GrassC3 -> 100.0 %
GrassC4 -> 0.0 %
Sedge -> 0.0 %
BdlEvgSh -> 0.0 %
BdlDCoSh -> 0.0 %


<xarray.DataArray 'fcancmx' (icc: 12)>
array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
Coordinates:
  * icc      (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
    lat      float64 73.15
    lon      float64 -80.0
    tile     int32 1
Attributes:
    units:      -
    long_name:  PFT fractional coverage per grid cell

### Set soil color

In [33]:
with xr.set_options(keep_attrs=True):
    ds['SOCI'][0, 0, 0] = SOCI
ds.SOCI

<xarray.DataArray 'SOCI' (tile: 1, lat: 1, lon: 1)>
array([[[6.]]])
Coordinates:
  * lat      (lat) float64 73.15
  * lon      (lon) float64 -80.0
  * tile     (tile) int32 1
Attributes:
    units:      index
    long_name:  Soil colour index

### Set permeable depth

In [34]:
with xr.set_options(keep_attrs=True):
    ds['SDEP'][0, 0, 0] = SDEP
ds.SDEP

<xarray.DataArray 'SDEP' (tile: 1, lat: 1, lon: 1)>
array([[[4.96000004]]])
Coordinates:
  * lat      (lat) float64 73.15
  * lon      (lon) float64 -80.0
  * tile     (tile) int32 1
Attributes:
    units:      m
    long_name:  Soil permeable depth

### Check maximum level before bedrock
https://gitlab.com/jormelton/classic/-/blob/develop/src/modelStateDrivers.f90?ref_type=heads#L968

In [35]:
ds.DELZ

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.2,
        0.3,  0.4,  0.5,  1. ,  3. ,  5. , 15. , 30. ,  5. ])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
Attributes:
    units:      m
    long_name:  Ground layer thickness

In [36]:
i = 0
while ds.DELZ.cumsum()[i] < SDEP:
    i += 1
    if i > ds.DELZ.shape[0]-1:
        print('The permable depth is greater than the model levels')
        break
        
maxlevel = i + 1 # first level of bedrock (next to the one containing SDEP)
maxlevel

16

In [37]:
ds.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9

In [38]:
ds.DELZ.cumsum()[maxlevel]

<xarray.DataArray 'DELZ' ()>
array(11.4)
Coordinates:
    layer    float64 8.9

In [39]:
ds.DELZ.cumsum()[:maxlevel]

<xarray.DataArray 'DELZ' (layer: 16)>
array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.2, 1.5, 1.9,
       2.4, 3.4, 6.4])
Coordinates:
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 1.35 1.7 2.15 2.9 4.9

In [40]:
ds.DELZ.cumsum()[maxlevel:]

<xarray.DataArray 'DELZ' (layer: 4)>
array([11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) float64 8.9 18.9 41.4 58.9

### Set soil contents and flags
Note: flags are only set in the SAND variable (-3 bedrock / -2 peatland)

In [41]:
with xr.set_options(keep_attrs=True):
    # Set values until bedrock
    ds.SAND[0, :maxlevel, 0, 0] = SAND[:maxlevel]
    ds.CLAY[0, :maxlevel, 0, 0] = CLAY[:maxlevel]
    ds.ORGM[0, :maxlevel, 0, 0] = ORGM[:maxlevel]
    
    # Set bedrock values
    ds.SAND[0, maxlevel:, 0, 0] = -3 # flag for bedrock
    ds.CLAY[0, maxlevel:, 0, 0] = 0.
    ds.ORGM[0, maxlevel:, 0, 0] = 0.

In [42]:
# Current values
ds.SAND[0, :, 0, 0]

<xarray.DataArray 'SAND' (layer: 20)>
array([47.6       , 43.5       , 63.5       , 54.44800568, 54.44800568,
       54.44800568, 54.68405151, 54.68405151, 54.68405151, 54.68405151,
       54.2089138 , 54.2089138 , 54.2089138 , 54.2089138 , 54.2089138 ,
       54.2089138 , -3.        , -3.        , -3.        , -3.        ])
Coordinates:
    lat      float64 73.15
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
    lon      float64 -80.0
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage sand content

In [43]:
ds.CLAY[0, :, 0, 0]

<xarray.DataArray 'CLAY' (layer: 20)>
array([0.        , 0.        , 0.1       , 7.54218798, 7.54218798,
       7.54218798, 7.64504318, 7.64504318, 7.64504318, 7.64504318,
       7.43632059, 7.43632059, 7.43632059, 7.43632059, 7.43632059,
       7.43632059, 0.        , 0.        , 0.        , 0.        ])
Coordinates:
    lat      float64 73.15
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
    lon      float64 -80.0
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage clay content

In [44]:
ds.ORGM[0, :, 0, 0]

<xarray.DataArray 'ORGM' (layer: 20)>
array([6.        , 1.5       , 1.5       , 6.46120691, 6.46120691,
       6.46120691, 6.07327604, 6.07327604, 6.07327604, 6.07327604,
       5.75718403, 5.75718403, 5.75718403, 5.75718403, 5.75718403,
       5.75718403, 0.        , 0.        , 0.        , 0.        ])
Coordinates:
    lat      float64 73.15
  * layer    (layer) float64 0.05 0.15 0.25 0.35 0.45 ... 4.9 8.9 18.9 41.4 58.9
    lon      float64 -80.0
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage organic matter content

In [45]:
ds

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 73.15
  * layer              (layer) float64 0.05 0.15 0.25 0.35 ... 18.9 41.4 58.9
  * lon                (lon) float64 -80.0
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 0.0 0.0 0.1 ... 0.0 0.0
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

## Save to netCDF
If CLASS would be run without CTEM further variables should be set. See: https://cccma.gitlab.io/classic/basicInputs.html -> Required vegetation data


In [46]:
ds.to_netcdf(path_out+'/'+site+'_init_spinup_'+exp+'.nc')
ds.to_netcdf(path_out+'/rsfile_spinup_'+exp+'.nc')

In [47]:
!mkdir -p /home/lalandmi/eccc/classic-develop/outputFiles/SnowArctic/{site}/spinup_{exp}